# Interpretable Feature Engineering for DPF Maintenance Prediction

This notebook teaches you how to create **explainable features** from raw sensor data that fleet managers can understand and trust.

## 🎯 Why Interpretable Features Matter

**Traditional Approach**: Complex features that work but can't be explained
- Fourier transforms, wavelet coefficients, deep learning embeddings
- Hard to explain why a vehicle needs maintenance
- Fleet managers can't take actionable steps

**Our Approach**: Simple, domain-driven features that tell a story
- "Engine load has been trending upward 5% per week"
- "DEF level drops below 75% more than usual"
- "Recent driving patterns 20% more aggressive than historical"

## 📚 What You'll Learn

1. **Trend Analysis**: How to detect meaningful changes in sensor patterns
2. **Threshold Engineering**: Creating actionable alert boundaries
3. **Pattern Recognition**: Identifying operational behavior changes
4. **Domain Knowledge Integration**: Using fleet management expertise in features
5. **Feature Validation**: Testing that features actually predict maintenance

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('default')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (12, 6)

print("🚀 Ready to build interpretable features!")
print("📚 This notebook will teach you to create features that:")
print("   • Fleet managers can understand")
print("   • Maintenance teams can act upon")
print("   • Actually predict DPF issues")

🚀 Ready to build interpretable features!
📚 This notebook will teach you to create features that:
   • Fleet managers can understand
   • Maintenance teams can act upon
   • Actually predict DPF issues


## 📊 Step 1: Understanding Raw Sensor Data

Before we engineer features, let's understand what our raw sensor data looks like and what it tells us about DPF health.

In [2]:
# Load our datasets
print("📁 Loading DPF datasets...")

try:
    maintenance_df = pd.read_csv('../data/dpf_maintenance_records.csv')
    sensor_df = pd.read_csv('../data/dpf_vehicle_stats.csv')
    
    # Convert datetime columns
    maintenance_df['Date of Issue'] = pd.to_datetime(maintenance_df['Date of Issue'])
    sensor_df['time'] = pd.to_datetime(sensor_df['time'])
    
    print(f"✅ Loaded {len(maintenance_df)} maintenance events")
    print(f"✅ Loaded {len(sensor_df):,} sensor readings")
    
except FileNotFoundError:
    print("❌ Please run the data munging script first: uv run python 01_data_munging.py")
    raise

📁 Loading DPF datasets...
✅ Loaded 82 maintenance events
✅ Loaded 4,466,272 sensor readings


In [3]:
# Identify DPF-relevant sensors and their characteristics
print("🔍 Analyzing DPF-Relevant Sensors")
print("="*50)

# Define sensors most relevant to DPF health with their meanings
dpf_sensors = {
    'engineLoadPercent': {
        'description': 'Engine Load Percentage',
        'dpf_relevance': 'High load generates more soot, stressing DPF',
        'normal_range': (20, 60),
        'alert_thresholds': {'high': 80, 'low': 10},
        'unit': '%'
    },
    'engineRpm': {
        'description': 'Engine RPM',
        'dpf_relevance': 'RPM affects DPF regeneration efficiency',
        'normal_range': (800, 1800),
        'alert_thresholds': {'high': 2000, 'low': 600},
        'unit': 'RPM'
    },
    'defLevelMilliPercent': {
        'description': 'DEF (Diesel Exhaust Fluid) Level',
        'dpf_relevance': 'Low DEF prevents proper DPF operation',
        'normal_range': (60000, 95000),  # 60-95%
        'alert_thresholds': {'high': 95000, 'low': 50000},
        'unit': 'milli-%'
    },
    'engineCoolantTemperatureMilliC': {
        'description': 'Engine Coolant Temperature',
        'dpf_relevance': 'Temperature affects DPF regeneration cycles',
        'normal_range': (75000, 90000),  # 75-90°C
        'alert_thresholds': {'high': 95000, 'low': 65000},
        'unit': 'milli-°C'
    },
    'ecuSpeedMph': {
        'description': 'Vehicle Speed',
        'dpf_relevance': 'Highway speeds help DPF regeneration',
        'normal_range': (0, 65),
        'alert_thresholds': {'high': 80, 'low': 0},
        'unit': 'mph'
    }
}

# Analyze data availability for each sensor
for sensor, info in dpf_sensors.items():
    if sensor in sensor_df.columns:
        non_null = sensor_df[sensor].notna().sum()
        total = len(sensor_df)
        availability = (non_null / total) * 100
        
        print(f"\n📊 {info['description']} ({sensor})")
        print(f"   📈 Data Availability: {availability:.1f}% ({non_null:,} readings)")
        print(f"   🎯 DPF Relevance: {info['dpf_relevance']}")
        print(f"   📏 Normal Range: {info['normal_range']} {info['unit']}")
        
        if non_null > 0:
            values = sensor_df[sensor].dropna()
            print(f"   📊 Current Range: {values.min():.1f} - {values.max():.1f} {info['unit']}")
    else:
        print(f"\n❌ {info['description']} ({sensor}) - No data available")

🔍 Analyzing DPF-Relevant Sensors

📊 Engine Load Percentage (engineLoadPercent)
   📈 Data Availability: 36.3% (1,619,223 readings)
   🎯 DPF Relevance: High load generates more soot, stressing DPF
   📏 Normal Range: (20, 60) %
   📊 Current Range: 1.0 - 125.0 %

📊 Engine RPM (engineRpm)
   📈 Data Availability: 42.3% (1,889,794 readings)
   🎯 DPF Relevance: RPM affects DPF regeneration efficiency
   📏 Normal Range: (800, 1800) RPM
   📊 Current Range: 26.0 - 2618.0 RPM

📊 DEF (Diesel Exhaust Fluid) Level (defLevelMilliPercent)
   📈 Data Availability: 45.9% (2,048,969 readings)
   🎯 DPF Relevance: Low DEF prevents proper DPF operation
   📏 Normal Range: (60000, 95000) milli-%
   📊 Current Range: 400.0 - 100000.0 milli-%

📊 Engine Coolant Temperature (engineCoolantTemperatureMilliC)
   📈 Data Availability: 33.5% (1,497,692 readings)
   🎯 DPF Relevance: Temperature affects DPF regeneration cycles
   📏 Normal Range: (75000, 90000) milli-°C
   📊 Current Range: 1000.0 - 123000.0 milli-°C

📊 Vehic

## 🔧 Step 2: Trend Analysis Features

**Trend features** help us understand if a sensor is getting worse over time. These are highly interpretable:
- **Positive slope**: "Engine load is increasing 2% per week"
- **Negative slope**: "DEF level is dropping 5% per week"
- **Flat trend**: "Temperature has been stable"

### Why This Works for Fleet Managers:
- Easy to visualize and understand
- Actionable: know which direction things are heading
- Predictive: trends often continue into the future

In [4]:
def calculate_trend_features(sensor_data, sensor_name, time_col='time'):
    """
    Calculate interpretable trend features for a sensor.
    Returns features that fleet managers can understand and act upon.
    """
    if len(sensor_data) < 3:  # Need minimum points for trend
        return {}
    
    # Prepare data
    data = sensor_data.dropna().sort_values(time_col)
    if len(data) < 3:
        return {}
    
    values = data[sensor_name].values
    times = pd.to_numeric(data[time_col]) / 1e9  # Convert to seconds
    
    features = {}
    
    # 1. LINEAR TREND (Most Important)
    # Calculate slope in original units per day
    slope_per_second, intercept = np.polyfit(times, values, 1)
    slope_per_day = slope_per_second * 86400  # Convert to per day
    
    features[f'{sensor_name}_trend_slope_per_day'] = slope_per_day
    
    # 2. TREND STRENGTH (How reliable is the trend?)
    # R-squared shows how well the linear trend fits
    correlation = np.corrcoef(times, values)[0, 1]
    r_squared = correlation ** 2 if not np.isnan(correlation) else 0
    features[f'{sensor_name}_trend_strength'] = r_squared
    
    # 3. TREND SIGNIFICANCE (Is this trend meaningful?)
    # Calculate statistical significance of the trend
    if len(values) >= 3:
        slope, intercept, r_value, p_value, std_err = stats.linregress(times, values)
        features[f'{sensor_name}_trend_p_value'] = p_value
        features[f'{sensor_name}_trend_significant'] = 1 if p_value < 0.05 else 0
    
    # 4. RECENT VS HISTORICAL TREND
    # Compare recent behavior to historical (last 25% vs first 75%)
    split_point = int(len(values) * 0.75)
    if split_point > 2 and split_point < len(values) - 2:
        historical_mean = np.mean(values[:split_point])
        recent_mean = np.mean(values[split_point:])
        
        if historical_mean != 0:
            percent_change = ((recent_mean - historical_mean) / abs(historical_mean)) * 100
            features[f'{sensor_name}_recent_vs_historical_pct'] = percent_change
    
    # 5. VOLATILITY TREND (Is stability changing?)
    # Calculate if the sensor is becoming more or less stable
    if len(values) >= 6:
        mid_point = len(values) // 2
        early_std = np.std(values[:mid_point])
        late_std = np.std(values[mid_point:])
        
        if early_std > 0:
            volatility_change = (late_std - early_std) / early_std
            features[f'{sensor_name}_volatility_change'] = volatility_change
    
    return features

# Demonstrate trend analysis with a sample vehicle
print("🔍 Demonstrating Trend Analysis")
print("="*40)

# Get a vehicle with good data coverage
vehicle_counts = sensor_df['vin'].value_counts()
if len(vehicle_counts) > 0:
    sample_vin = vehicle_counts.index[0]
    sample_data = sensor_df[sensor_df['vin'] == sample_vin].copy()
    
    print(f"Sample Vehicle: {sample_vin}")
    print(f"Data Points: {len(sample_data)}")
    print(f"Date Range: {sample_data['time'].min()} to {sample_data['time'].max()}")
    
    # Calculate trends for each DPF sensor
    all_trends = {}
    for sensor in dpf_sensors.keys():
        if sensor in sample_data.columns:
            sensor_subset = sample_data[['time', sensor]].dropna()
            if len(sensor_subset) >= 5:
                trends = calculate_trend_features(sensor_subset, sensor)
                all_trends.update(trends)
                
                # Show human-readable interpretation
                slope_key = f'{sensor}_trend_slope_per_day'
                if slope_key in trends:
                    slope = trends[slope_key]
                    direction = "increasing" if slope > 0 else "decreasing"
                    strength = trends.get(f'{sensor}_trend_strength', 0)
                    
                    print(f"\n📈 {dpf_sensors[sensor]['description']}:")
                    print(f"   Trend: {direction} by {abs(slope):.3f} {dpf_sensors[sensor]['unit']}/day")
                    print(f"   Strength: {strength:.3f} (0=weak, 1=strong linear trend)")
                    
                    # Interpret for fleet manager
                    if abs(slope) > 0.1 and strength > 0.3:
                        print(f"   🚨 Alert: Strong {direction} trend detected!")
                    elif abs(slope) > 0.05:
                        print(f"   ⚠️ Caution: Moderate {direction} trend")
                    else:
                        print(f"   ✅ Normal: Stable readings")
    
    print(f"\n📊 Total trend features calculated: {len(all_trends)}")
else:
    print("❌ No vehicle data available for trend analysis")

🔍 Demonstrating Trend Analysis
Sample Vehicle: 1XK1D40X1NJ495537
Data Points: 269964
Date Range: 2024-06-08 00:03:00+00:00 to 2025-05-21 20:40:00+00:00

📈 Engine Load Percentage:
   Trend: increasing by 0.013 %/day
   Strength: 0.001 (0=weak, 1=strong linear trend)
   ✅ Normal: Stable readings

📈 Engine RPM:
   Trend: increasing by 0.111 RPM/day
   Strength: 0.001 (0=weak, 1=strong linear trend)
   ⚠️ Caution: Moderate increasing trend

📈 DEF (Diesel Exhaust Fluid) Level:
   Trend: decreasing by 61.047 milli-%/day
   Strength: 0.072 (0=weak, 1=strong linear trend)
   ⚠️ Caution: Moderate decreasing trend

📈 Engine Coolant Temperature:
   Trend: decreasing by 9.565 milli-°C/day
   Strength: 0.005 (0=weak, 1=strong linear trend)
   ⚠️ Caution: Moderate decreasing trend

📈 Vehicle Speed:
   Trend: decreasing by 0.003 mph/day
   Strength: 0.000 (0=weak, 1=strong linear trend)
   ✅ Normal: Stable readings

📊 Total trend features calculated: 30


## 🎯 Step 3: Threshold-Based Features

**Threshold features** tell us how often a sensor operates outside normal ranges. These are immediately actionable:
- "Engine runs above 80% load 25% of the time" → Reduce load
- "DEF level below 60% for 3 days" → Refill DEF
- "Temperature exceeds 95°C twice this week" → Check cooling system

### Fleet Manager Benefits:
- Clear operational boundaries
- Direct maintenance actions
- Easy to set up monitoring alerts

In [5]:
def calculate_threshold_features(sensor_data, sensor_name, thresholds, time_col='time'):
    """
    Calculate threshold-based features that are easy to interpret and act upon.
    """
    if len(sensor_data) < 2:
        return {}
    
    # Prepare data
    data = sensor_data.dropna().sort_values(time_col)
    if len(data) < 2:
        return {}
    
    values = data[sensor_name].values
    features = {}
    
    # 1. PERCENTAGE OF TIME OUTSIDE THRESHOLDS
    
    # Time above high threshold
    if 'high' in thresholds:
        high_thresh = thresholds['high']
        pct_above_high = (values > high_thresh).mean() * 100
        features[f'{sensor_name}_pct_time_above_high'] = pct_above_high
    
    # Time below low threshold
    if 'low' in thresholds:
        low_thresh = thresholds['low']
        pct_below_low = (values < low_thresh).mean() * 100
        features[f'{sensor_name}_pct_time_below_low'] = pct_below_low
    
    # Total time outside normal range
    if 'high' in thresholds and 'low' in thresholds:
        outside_normal = ((values > thresholds['high']) | (values < thresholds['low'])).mean() * 100
        features[f'{sensor_name}_pct_time_outside_normal'] = outside_normal
    
    # 2. CONSECUTIVE VIOLATIONS (Streak Analysis)
    
    # Longest streak above high threshold
    if 'high' in thresholds:
        above_high = values > thresholds['high']
        max_streak_high = calculate_max_consecutive(above_high)
        features[f'{sensor_name}_max_streak_above_high'] = max_streak_high
    
    # Longest streak below low threshold
    if 'low' in thresholds:
        below_low = values < thresholds['low']
        max_streak_low = calculate_max_consecutive(below_low)
        features[f'{sensor_name}_max_streak_below_low'] = max_streak_low
    
    # 3. FREQUENCY OF VIOLATIONS
    
    # How many separate incidents of threshold violations?
    if 'high' in thresholds:
        high_violations = count_violation_episodes(values > thresholds['high'])
        features[f'{sensor_name}_high_violation_episodes'] = high_violations
    
    if 'low' in thresholds:
        low_violations = count_violation_episodes(values < thresholds['low'])
        features[f'{sensor_name}_low_violation_episodes'] = low_violations
    
    # 4. SEVERITY OF VIOLATIONS
    
    # Average severity when violations occur
    if 'high' in thresholds:
        high_violations_mask = values > thresholds['high']
        if high_violations_mask.any():
            avg_excess = (values[high_violations_mask] - thresholds['high']).mean()
            features[f'{sensor_name}_avg_excess_above_high'] = avg_excess
    
    if 'low' in thresholds:
        low_violations_mask = values < thresholds['low']
        if low_violations_mask.any():
            avg_deficit = (thresholds['low'] - values[low_violations_mask]).mean()
            features[f'{sensor_name}_avg_deficit_below_low'] = avg_deficit
    
    return features

def calculate_max_consecutive(boolean_array):
    """Calculate maximum consecutive True values in a boolean array."""
    if len(boolean_array) == 0:
        return 0
    
    max_streak = 0
    current_streak = 0
    
    for value in boolean_array:
        if value:
            current_streak += 1
            max_streak = max(max_streak, current_streak)
        else:
            current_streak = 0
    
    return max_streak

def count_violation_episodes(boolean_array):
    """Count number of separate violation episodes (groups of consecutive True values)."""
    if len(boolean_array) == 0:
        return 0
    
    episodes = 0
    in_violation = False
    
    for value in boolean_array:
        if value and not in_violation:
            episodes += 1
            in_violation = True
        elif not value:
            in_violation = False
    
    return episodes

# Demonstrate threshold analysis
print("🎯 Demonstrating Threshold Analysis")
print("="*40)

if len(vehicle_counts) > 0:
    sample_vin = vehicle_counts.index[0]
    sample_data = sensor_df[sensor_df['vin'] == sample_vin].copy()
    
    print(f"Analyzing thresholds for Vehicle: {sample_vin}")
    
    # Calculate threshold features for each sensor
    all_threshold_features = {}
    
    for sensor, info in dpf_sensors.items():
        if sensor in sample_data.columns:
            sensor_subset = sample_data[['time', sensor]].dropna()
            if len(sensor_subset) >= 5:
                thresholds = info['alert_thresholds']
                threshold_features = calculate_threshold_features(sensor_subset, sensor, thresholds)
                all_threshold_features.update(threshold_features)
                
                print(f"\n🎯 {info['description']} ({sensor}):")
                
                # Show key threshold metrics
                pct_high_key = f'{sensor}_pct_time_above_high'
                pct_low_key = f'{sensor}_pct_time_below_low'
                
                if pct_high_key in threshold_features:
                    pct_high = threshold_features[pct_high_key]
                    print(f"   ⬆️ Time above {thresholds['high']} {info['unit']}: {pct_high:.1f}%")
                    
                    if pct_high > 20:
                        print(f"      🚨 ALERT: Excessive high readings!")
                    elif pct_high > 10:
                        print(f"      ⚠️ WARNING: Frequent high readings")
                    else:
                        print(f"      ✅ Normal high threshold behavior")
                
                if pct_low_key in threshold_features:
                    pct_low = threshold_features[pct_low_key]
                    print(f"   ⬇️ Time below {thresholds['low']} {info['unit']}: {pct_low:.1f}%")
                    
                    if pct_low > 20:
                        print(f"      🚨 ALERT: Excessive low readings!")
                    elif pct_low > 10:
                        print(f"      ⚠️ WARNING: Frequent low readings")
                    else:
                        print(f"      ✅ Normal low threshold behavior")
                
                # Show streak information
                streak_high_key = f'{sensor}_max_streak_above_high'
                if streak_high_key in threshold_features:
                    streak = threshold_features[streak_high_key]
                    print(f"   📊 Longest high streak: {streak} consecutive readings")
    
    print(f"\n📊 Total threshold features calculated: {len(all_threshold_features)}")
else:
    print("❌ No vehicle data available for threshold analysis")

🎯 Demonstrating Threshold Analysis
Analyzing thresholds for Vehicle: 1XK1D40X1NJ495537

🎯 Engine Load Percentage (engineLoadPercent):
   ⬆️ Time above 80 %: 24.8%
      🚨 ALERT: Excessive high readings!
   ⬇️ Time below 10 %: 6.4%
      ✅ Normal low threshold behavior
   📊 Longest high streak: 16 consecutive readings

🎯 Engine RPM (engineRpm):
   ⬆️ Time above 2000 RPM: 0.0%
      ✅ Normal high threshold behavior
   ⬇️ Time below 600 RPM: 4.8%
      ✅ Normal low threshold behavior
   📊 Longest high streak: 1 consecutive readings

🎯 DEF (Diesel Exhaust Fluid) Level (defLevelMilliPercent):
   ⬆️ Time above 95000 milli-%: 5.4%
      ✅ Normal high threshold behavior
   ⬇️ Time below 50000 milli-%: 28.5%
      🚨 ALERT: Excessive low readings!
   📊 Longest high streak: 101 consecutive readings

🎯 Engine Coolant Temperature (engineCoolantTemperatureMilliC):
   ⬆️ Time above 95000 milli-°C: 17.6%
      ⚠️ WARNING: Frequent high readings
   ⬇️ Time below 65000 milli-°C: 4.2%
      ✅ Normal low 

## 🔄 Step 4: Operational Pattern Features

**Pattern features** capture changes in how the vehicle is being operated. These help identify external factors affecting DPF health:

- **Duty Cycle Changes**: "Vehicle now works 30% harder than usual"
- **Drive Profile Shifts**: "More city driving, less highway (bad for DPF)"
- **Load Distribution**: "Consistently heavy loads vs mixed operation"

### Why This Matters:
- Operational changes often precede maintenance needs
- Helps identify root causes (driver behavior, route changes)
- Enables proactive operational adjustments

In [6]:
def calculate_operational_pattern_features(sensor_data, time_col='time'):
    """
    Calculate operational pattern features that help identify changes in vehicle usage.
    These features help fleet managers understand if operational changes are affecting DPF health.
    """
    if len(sensor_data) < 10:  # Need reasonable amount of data for patterns
        return {}
    
    features = {}
    
    # 1. DUTY CYCLE ANALYSIS (How hard is the vehicle working?)
    if 'engineLoadPercent' in sensor_data.columns:
        load_data = sensor_data['engineLoadPercent'].dropna()
        if len(load_data) >= 10:
            
            # Average duty cycle
            avg_load = load_data.mean()
            features['avg_engine_load'] = avg_load
            
            # Load distribution analysis
            features['pct_time_high_load'] = (load_data > 70).mean() * 100  # >70% load
            features['pct_time_medium_load'] = ((load_data >= 40) & (load_data <= 70)).mean() * 100
            features['pct_time_low_load'] = (load_data < 40).mean() * 100  # <40% load
            
            # Load consistency (coefficient of variation)
            if avg_load > 0:
                features['load_consistency'] = load_data.std() / avg_load
    
    # 2. DRIVE PROFILE ANALYSIS (City vs Highway driving)
    if 'ecuSpeedMph' in sensor_data.columns:
        speed_data = sensor_data['ecuSpeedMph'].dropna()
        if len(speed_data) >= 10:
            
            # Speed distribution (proxy for drive cycle)
            features['pct_time_highway_speed'] = (speed_data > 45).mean() * 100  # >45 mph = highway
            features['pct_time_city_speed'] = ((speed_data > 0) & (speed_data <= 35)).mean() * 100  # 0-35 mph = city
            features['pct_time_stopped'] = (speed_data == 0).mean() * 100  # Stopped
            
            # Average operating speed
            features['avg_operating_speed'] = speed_data[speed_data > 0].mean() if (speed_data > 0).any() else 0
            
            # Speed variability (smooth vs stop-and-go driving)
            features['speed_variability'] = speed_data.std()
    
    # 3. OPERATIONAL TIMING PATTERNS
    if time_col in sensor_data.columns:
        # Add hour of day for operational pattern analysis
        sensor_data_copy = sensor_data.copy()
        sensor_data_copy['hour'] = pd.to_datetime(sensor_data_copy[time_col]).dt.hour
        
        # Operating hours distribution
        operating_hours = sensor_data_copy['hour'].value_counts().sort_index()
        
        # Peak operating hours
        if len(operating_hours) > 0:
            features['peak_operating_hour'] = operating_hours.idxmax()
            features['operating_hour_spread'] = operating_hours.max() - operating_hours.min()
    
    # 4. MULTI-SENSOR OPERATIONAL PATTERNS
    # Correlation between load and speed (operational efficiency)
    if 'engineLoadPercent' in sensor_data.columns and 'ecuSpeedMph' in sensor_data.columns:
        load_speed_data = sensor_data[['engineLoadPercent', 'ecuSpeedMph']].dropna()
        if len(load_speed_data) >= 10:
            correlation = load_speed_data['engineLoadPercent'].corr(load_speed_data['ecuSpeedMph'])
            features['load_speed_correlation'] = correlation if not np.isnan(correlation) else 0
    
    # Engine efficiency patterns (RPM vs Load)
    if 'engineRpm' in sensor_data.columns and 'engineLoadPercent' in sensor_data.columns:
        rpm_load_data = sensor_data[['engineRpm', 'engineLoadPercent']].dropna()
        if len(rpm_load_data) >= 10:
            # Calculate efficiency proxy (load per RPM)
            efficiency_data = rpm_load_data[rpm_load_data['engineRpm'] > 0].copy()
            if len(efficiency_data) > 0:
                efficiency_data['load_per_rpm'] = efficiency_data['engineLoadPercent'] / efficiency_data['engineRpm'] * 1000
                features['avg_load_per_rpm'] = efficiency_data['load_per_rpm'].mean()
    
    return features

def compare_operational_periods(sensor_data, split_ratio=0.7, time_col='time'):
    """
    Compare operational patterns between historical (early) and recent periods.
    This helps identify if operational changes are affecting DPF health.
    """
    if len(sensor_data) < 20:  # Need enough data to split meaningfully
        return {}
    
    # Sort by time and split
    sorted_data = sensor_data.sort_values(time_col)
    split_point = int(len(sorted_data) * split_ratio)
    
    historical_data = sorted_data.iloc[:split_point]
    recent_data = sorted_data.iloc[split_point:]
    
    # Calculate operational features for both periods
    historical_features = calculate_operational_pattern_features(historical_data, time_col)
    recent_features = calculate_operational_pattern_features(recent_data, time_col)
    
    # Calculate changes
    changes = {}
    for feature in historical_features:
        if feature in recent_features:
            historical_value = historical_features[feature]
            recent_value = recent_features[feature]
            
            if historical_value != 0:
                percent_change = ((recent_value - historical_value) / abs(historical_value)) * 100
                changes[f'{feature}_change_pct'] = percent_change
    
    return changes

# Demonstrate operational pattern analysis
print("🔄 Demonstrating Operational Pattern Analysis")
print("="*50)

if len(vehicle_counts) > 0:
    sample_vin = vehicle_counts.index[0]
    sample_data = sensor_df[sensor_df['vin'] == sample_vin].copy()
    
    print(f"Analyzing operational patterns for Vehicle: {sample_vin}")
    print(f"Data points: {len(sample_data)}")
    
    # Calculate current operational features
    operational_features = calculate_operational_pattern_features(sample_data)
    
    if operational_features:
        print(f"\n📊 Current Operational Profile:")
        
        # Duty cycle analysis
        if 'avg_engine_load' in operational_features:
            avg_load = operational_features['avg_engine_load']
            print(f"   🔧 Average Engine Load: {avg_load:.1f}%")
            
            if avg_load > 60:
                print(f"      🚨 High duty cycle - may stress DPF")
            elif avg_load < 30:
                print(f"      ⚠️ Low duty cycle - may not regenerate DPF properly")
            else:
                print(f"      ✅ Moderate duty cycle")
        
        # Drive profile analysis
        if 'pct_time_highway_speed' in operational_features:
            highway_pct = operational_features['pct_time_highway_speed']
            city_pct = operational_features.get('pct_time_city_speed', 0)
            
            print(f"   🛣️ Highway driving: {highway_pct:.1f}% of time")
            print(f"   🏙️ City driving: {city_pct:.1f}% of time")
            
            if highway_pct < 20:
                print(f"      🚨 Mostly city driving - poor for DPF regeneration")
            elif highway_pct > 60:
                print(f"      ✅ Good highway driving - helps DPF regeneration")
            else:
                print(f"      ⚠️ Mixed driving profile")
    
    # Compare historical vs recent patterns
    pattern_changes = compare_operational_periods(sample_data)
    
    if pattern_changes:
        print(f"\n📈 Operational Changes (Recent vs Historical):")
        
        significant_changes = {k: v for k, v in pattern_changes.items() if abs(v) > 10}
        
        if significant_changes:
            for change_feature, percent_change in significant_changes.items():
                base_feature = change_feature.replace('_change_pct', '')
                direction = "increased" if percent_change > 0 else "decreased"
                print(f"   📊 {base_feature}: {direction} by {abs(percent_change):.1f}%")
                
                # Interpret the change
                if abs(percent_change) > 25:
                    print(f"      🚨 Significant operational change detected!")
                elif abs(percent_change) > 15:
                    print(f"      ⚠️ Notable operational change")
        else:
            print(f"   ✅ Stable operational patterns")
    
    total_features = len(operational_features) + len(pattern_changes)
    print(f"\n📊 Total operational features calculated: {total_features}")
else:
    print("❌ No vehicle data available for operational pattern analysis")

🔄 Demonstrating Operational Pattern Analysis
Analyzing operational patterns for Vehicle: 1XK1D40X1NJ495537
Data points: 269964

📊 Current Operational Profile:
   🔧 Average Engine Load: 51.1%
      ✅ Moderate duty cycle
   🛣️ Highway driving: 73.6% of time
   🏙️ City driving: 17.2% of time
      ✅ Good highway driving - helps DPF regeneration

📈 Operational Changes (Recent vs Historical):
   📊 peak_operating_hour: increased by 16.7%
      ⚠️ Notable operational change
   📊 operating_hour_spread: decreased by 57.4%
      🚨 Significant operational change detected!

📊 Total operational features calculated: 28


## 🧪 Step 5: Feature Validation and Selection

Now let's validate that our interpretable features actually predict DPF maintenance needs. This step is crucial for building trust with fleet managers.

### What Makes a Good Predictive Feature:
1. **Statistically Significant**: Reliably different between healthy and failing vehicles
2. **Practically Meaningful**: Changes that fleet managers can act upon
3. **Stable**: Consistent behavior across different vehicles and time periods
4. **Explainable**: Clear causal relationship to DPF health

In [7]:
def build_comprehensive_feature_dataset(maintenance_df, sensor_df, window_days=30):
    """
    Build a comprehensive dataset with all our interpretable features.
    This combines trend, threshold, and operational pattern features.
    """
    print(f"🔧 Building comprehensive feature dataset...")
    
    feature_data = []
    
    for _, maintenance_row in maintenance_df.iterrows():
        vin = maintenance_row['VIN Number']
        maintenance_date = maintenance_row['Date of Issue']
        
        if pd.isna(vin) or pd.isna(maintenance_date):
            continue
        
        # Get sensor data before maintenance
        start_date = maintenance_date - timedelta(days=window_days)
        
        vehicle_data = sensor_df[
            (sensor_df['vin'] == vin) &
            (sensor_df['time'] >= start_date) &
            (sensor_df['time'] < maintenance_date)
        ].copy()
        
        if len(vehicle_data) < 10:  # Need minimum data
            continue
        
        # Calculate all feature types
        all_features = {
            'vin': vin,
            'vehicle_number': maintenance_row['Vehicle_Number'],
            'maintenance_date': maintenance_date,
            'maintenance_type': maintenance_row['lines_jobDescriptions'],
            'data_points': len(vehicle_data),
            'window_days': window_days
        }
        
        # 1. Trend features for each DPF sensor
        for sensor in dpf_sensors.keys():
            if sensor in vehicle_data.columns:
                sensor_subset = vehicle_data[['time', sensor]].dropna()
                if len(sensor_subset) >= 5:
                    trend_features = calculate_trend_features(sensor_subset, sensor)
                    all_features.update(trend_features)
        
        # 2. Threshold features for each DPF sensor
        for sensor, info in dpf_sensors.items():
            if sensor in vehicle_data.columns:
                sensor_subset = vehicle_data[['time', sensor]].dropna()
                if len(sensor_subset) >= 5:
                    threshold_features = calculate_threshold_features(
                        sensor_subset, sensor, info['alert_thresholds']
                    )
                    all_features.update(threshold_features)
        
        # 3. Operational pattern features
        operational_features = calculate_operational_pattern_features(vehicle_data)
        all_features.update(operational_features)
        
        # 4. Operational change features
        pattern_changes = compare_operational_periods(vehicle_data)
        all_features.update(pattern_changes)
        
        feature_data.append(all_features)
    
    return pd.DataFrame(feature_data)

# Build the comprehensive feature dataset
print("🔧 Building Comprehensive Interpretable Feature Dataset")
print("="*60)

feature_dataset = build_comprehensive_feature_dataset(maintenance_df, sensor_df)

if len(feature_dataset) > 0:
    print(f"✅ Built feature dataset with {len(feature_dataset)} examples")
    
    # Analyze feature coverage
    feature_cols = [col for col in feature_dataset.columns if col not in [
        'vin', 'vehicle_number', 'maintenance_date', 'maintenance_type', 'data_points', 'window_days'
    ]]
    
    print(f"📊 Total interpretable features: {len(feature_cols)}")
    
    # Categorize features by type
    trend_features = [col for col in feature_cols if 'trend' in col]
    threshold_features = [col for col in feature_cols if any(x in col for x in ['pct_time', 'streak', 'violation', 'excess', 'deficit'])]
    operational_features = [col for col in feature_cols if col not in trend_features + threshold_features]
    
    print(f"   📈 Trend features: {len(trend_features)}")
    print(f"   🎯 Threshold features: {len(threshold_features)}")
    print(f"   🔄 Operational features: {len(operational_features)}")
    
    # Show maintenance type distribution
    print(f"\n🔧 Maintenance Type Distribution:")
    for maint_type, count in feature_dataset['maintenance_type'].value_counts().items():
        print(f"   {maint_type}: {count} examples")
        
else:
    print("❌ Could not build feature dataset - insufficient data")

🔧 Building Comprehensive Interpretable Feature Dataset
🔧 Building comprehensive feature dataset...


TypeError: Invalid comparison between dtype=datetime64[ns, UTC] and Timestamp

In [ ]:
# Validate feature quality and predictive power
def validate_interpretable_features(feature_dataset):
    """
    Validate the quality and predictive power of our interpretable features.
    """
    if len(feature_dataset) == 0:
        return
    
    print("🧪 Validating Interpretable Features")
    print("="*40)
    
    # Get feature columns
    feature_cols = [col for col in feature_dataset.columns if col not in [
        'vin', 'vehicle_number', 'maintenance_date', 'maintenance_type', 'data_points', 'window_days'
    ]]
    
    if len(feature_cols) == 0:
        print("❌ No features to validate")
        return
    
    # 1. DATA QUALITY VALIDATION
    print("📊 Data Quality Analysis:")
    
    # Check for missing values
    missing_rates = feature_dataset[feature_cols].isnull().mean() * 100
    high_missing = missing_rates[missing_rates > 50]
    
    if len(high_missing) > 0:
        print(f"   ⚠️ Features with >50% missing data: {len(high_missing)}")
        for feature, missing_pct in high_missing.head(5).items():
            print(f"      - {feature}: {missing_pct:.1f}% missing")
    else:
        print(f"   ✅ Good data coverage - no features with >50% missing data")
    
    # 2. FEATURE DISCRIMINATIVE POWER
    print(f"\n🔍 Feature Discriminative Power (by Maintenance Type):")
    
    # For each maintenance type, find features that discriminate well
    maintenance_types = feature_dataset['maintenance_type'].unique()
    
    if len(maintenance_types) > 1:
        # Calculate feature means by maintenance type
        type_means = feature_dataset.groupby('maintenance_type')[feature_cols].mean()
        
        # Calculate coefficient of variation across maintenance types for each feature
        discriminative_power = {}
        for feature in feature_cols:
            values = type_means[feature].dropna()
            if len(values) > 1 and values.std() > 0:
                cv = values.std() / abs(values.mean()) if values.mean() != 0 else 0
                discriminative_power[feature] = cv
        
        # Show top discriminative features
        if discriminative_power:
            top_discriminative = sorted(discriminative_power.items(), key=lambda x: x[1], reverse=True)[:10]
            
            print(f"   🏆 Top 10 Most Discriminative Features:")
            for i, (feature, cv) in enumerate(top_discriminative, 1):
                # Determine feature type
                if 'trend' in feature:
                    feature_type = "📈 Trend"
                elif any(x in feature for x in ['pct_time', 'streak', 'violation']):
                    feature_type = "🎯 Threshold"
                else:
                    feature_type = "🔄 Operational"
                
                print(f"      {i}. {feature_type} - {feature}: CV={cv:.3f}")
    
    # 3. FEATURE INTERPRETABILITY ASSESSMENT
    print(f"\n💡 Feature Interpretability Assessment:")
    
    interpretability_scores = {
        'trend_slope': {'score': 10, 'explanation': 'Clear direction and rate of change'},
        'pct_time': {'score': 9, 'explanation': 'Percentage time outside thresholds'},
        'avg_': {'score': 8, 'explanation': 'Simple average values'},
        'max_streak': {'score': 8, 'explanation': 'Consecutive violations count'},
        'change_pct': {'score': 7, 'explanation': 'Percentage change over time'},
        'correlation': {'score': 6, 'explanation': 'Relationship between sensors'},
        'volatility': {'score': 5, 'explanation': 'Stability measure'}
    }
    
    # Score each feature
    feature_scores = []
    for feature in feature_cols:
        score = 3  # Default score
        explanation = "General metric"
        
        for pattern, info in interpretability_scores.items():
            if pattern in feature:
                score = info['score']
                explanation = info['explanation']
                break
        
        feature_scores.append((feature, score, explanation))
    
    # Show most interpretable features
    feature_scores.sort(key=lambda x: x[1], reverse=True)
    
    print(f"   🌟 Most Interpretable Features (Score 8-10):")
    high_interpretability = [f for f in feature_scores if f[1] >= 8]
    
    for feature, score, explanation in high_interpretability[:10]:
        print(f"      • {feature} (Score: {score}/10) - {explanation}")
    
    # 4. FEATURE RECOMMENDATION
    print(f"\n🎯 Feature Recommendations for Fleet Managers:")
    
    # Combine discriminative power and interpretability
    if discriminative_power:
        recommended_features = []
        
        for feature, interp_score, explanation in feature_scores:
            if feature in discriminative_power and interp_score >= 7:
                combined_score = discriminative_power[feature] * (interp_score / 10)
                recommended_features.append((feature, combined_score, interp_score, explanation))
        
        recommended_features.sort(key=lambda x: x[1], reverse=True)
        
        print(f"   🏆 Top 5 Recommended Features (High interpretability + High predictive power):")
        for i, (feature, combined_score, interp_score, explanation) in enumerate(recommended_features[:5], 1):
            print(f"      {i}. {feature}")
            print(f"         Interpretability: {interp_score}/10 - {explanation}")
            print(f"         Combined Score: {combined_score:.3f}")
            print()

# Run feature validation
if len(feature_dataset) > 0:
    validate_interpretable_features(feature_dataset)
else:
    print("❌ No feature dataset available for validation")

## 📋 Step 6: Feature Engineering Best Practices Summary

### 🎯 Key Principles for Interpretable Features:

1. **Domain-Driven**: Every feature should relate to known DPF failure modes
2. **Actionable**: Fleet managers should know what to do when a feature triggers
3. **Explainable**: Each feature should tell a clear story about vehicle health
4. **Validated**: Features must actually predict maintenance needs

### 📊 Feature Categories We've Built:

**Trend Features** (📈):
- Sensor slopes (increasing/decreasing patterns)
- Trend strength (how reliable the trend is)
- Recent vs historical comparisons

**Threshold Features** (🎯):
- Time spent outside normal ranges
- Consecutive violation streaks
- Severity of threshold breaches

**Operational Features** (🔄):
- Duty cycle changes
- Drive profile shifts (city vs highway)
- Multi-sensor pattern correlations

### 🚀 Implementation Roadmap:

1. **Start Simple**: Begin with top 5 most interpretable features
2. **Validate Continuously**: Track feature performance against actual maintenance
3. **Refine Thresholds**: Adjust based on operational experience
4. **Add Domain Knowledge**: Incorporate fleet manager insights into features
5. **Scale Gradually**: Add more features as confidence builds

In [ ]:
# Save the interpretable feature dataset for use in other notebooks
if len(feature_dataset) > 0:
    output_file = '../data/interpretable_features_dataset.csv'
    feature_dataset.to_csv(output_file, index=False)
    
    print("💾 FEATURE ENGINEERING COMPLETE!")
    print("="*50)
    print(f"✅ Saved interpretable features to: {output_file}")
    print(f"📊 Dataset contains: {len(feature_dataset)} examples")
    
    feature_cols = [col for col in feature_dataset.columns if col not in [
        'vin', 'vehicle_number', 'maintenance_date', 'maintenance_type', 'data_points', 'window_days'
    ]]
    print(f"🔧 Total features: {len(feature_cols)}")
    
    # Feature breakdown
    trend_features = [col for col in feature_cols if 'trend' in col]
    threshold_features = [col for col in feature_cols if any(x in col for x in ['pct_time', 'streak', 'violation'])]
    operational_features = [col for col in feature_cols if col not in trend_features + threshold_features]
    
    print(f"   📈 Trend features: {len(trend_features)}")
    print(f"   🎯 Threshold features: {len(threshold_features)}")
    print(f"   🔄 Operational features: {len(operational_features)}")
    
    print(f"\n🎉 Ready for model building and deployment!")
    print(f"Next step: Use these features in the explainable model notebook")
else:
    print("❌ No interpretable features dataset created")
    print("Please check data availability and try again")

print(f"\n📚 What You've Learned:")
print(f"• How to create trend features that show sensor degradation patterns")
print(f"• How to build threshold features for actionable alerts")
print(f"• How to detect operational pattern changes affecting DPF health")
print(f"• How to validate feature quality and interpretability")
print(f"• Best practices for explainable feature engineering")

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('default')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (12, 6)

print("🚀 Ready to build interpretable features!")
print("📚 This notebook will teach you to create features that:")
print("   • Fleet managers can understand")
print("   • Maintenance teams can act upon")
print("   • Actually predict DPF issues")

🚀 Ready to build interpretable features!
📚 This notebook will teach you to create features that:
   • Fleet managers can understand
   • Maintenance teams can act upon
   • Actually predict DPF issues


In [9]:
# Load our datasets
print("📁 Loading DPF datasets...")

try:
    maintenance_df = pd.read_csv('../data/dpf_maintenance_records.csv')
    sensor_df = pd.read_csv('../data/dpf_vehicle_stats.csv')
    
    # Convert datetime columns and handle timezones
    maintenance_df['Date of Issue'] = pd.to_datetime(maintenance_df['Date of Issue'])
    sensor_df['time'] = pd.to_datetime(sensor_df['time']).dt.tz_localize(None)  # Remove timezone info
    
    print(f"✅ Loaded {len(maintenance_df)} maintenance events")
    print(f"✅ Loaded {len(sensor_df):,} sensor readings")
    
except FileNotFoundError:
    print("❌ Please run the data munging script first: uv run python 01_data_munging.py")
    raise

📁 Loading DPF datasets...
✅ Loaded 82 maintenance events
✅ Loaded 4,466,272 sensor readings


In [10]:
# Identify DPF-relevant sensors and their characteristics
print("🔍 Analyzing DPF-Relevant Sensors")
print("="*50)

# Define sensors most relevant to DPF health with their meanings
dpf_sensors = {
    'engineLoadPercent': {
        'description': 'Engine Load Percentage',
        'dpf_relevance': 'High load generates more soot, stressing DPF',
        'normal_range': (20, 60),
        'alert_thresholds': {'high': 80, 'low': 10},
        'unit': '%'
    },
    'engineRpm': {
        'description': 'Engine RPM',
        'dpf_relevance': 'RPM affects DPF regeneration efficiency',
        'normal_range': (800, 1800),
        'alert_thresholds': {'high': 2000, 'low': 600},
        'unit': 'RPM'
    },
    'defLevelMilliPercent': {
        'description': 'DEF (Diesel Exhaust Fluid) Level',
        'dpf_relevance': 'Low DEF prevents proper DPF operation',
        'normal_range': (60000, 95000),  # 60-95%
        'alert_thresholds': {'high': 95000, 'low': 50000},
        'unit': 'milli-%'
    },
    'engineCoolantTemperatureMilliC': {
        'description': 'Engine Coolant Temperature',
        'dpf_relevance': 'Temperature affects DPF regeneration cycles',
        'normal_range': (75000, 90000),  # 75-90°C
        'alert_thresholds': {'high': 95000, 'low': 65000},
        'unit': 'milli-°C'
    },
    'ecuSpeedMph': {
        'description': 'Vehicle Speed',
        'dpf_relevance': 'Highway speeds help DPF regeneration',
        'normal_range': (0, 65),
        'alert_thresholds': {'high': 80, 'low': 0},
        'unit': 'mph'
    }
}

# Analyze data availability for each sensor
for sensor, info in dpf_sensors.items():
    if sensor in sensor_df.columns:
        non_null = sensor_df[sensor].notna().sum()
        total = len(sensor_df)
        availability = (non_null / total) * 100
        
        print(f"\n📊 {info['description']} ({sensor})")
        print(f"   📈 Data Availability: {availability:.1f}% ({non_null:,} readings)")
        print(f"   🎯 DPF Relevance: {info['dpf_relevance']}")
        print(f"   📏 Normal Range: {info['normal_range']} {info['unit']}")
        
        if non_null > 0:
            values = sensor_df[sensor].dropna()
            print(f"   📊 Current Range: {values.min():.1f} - {values.max():.1f} {info['unit']}")
    else:
        print(f"\n❌ {info['description']} ({sensor}) - No data available")

🔍 Analyzing DPF-Relevant Sensors

📊 Engine Load Percentage (engineLoadPercent)
   📈 Data Availability: 36.3% (1,619,223 readings)
   🎯 DPF Relevance: High load generates more soot, stressing DPF
   📏 Normal Range: (20, 60) %
   📊 Current Range: 1.0 - 125.0 %

📊 Engine RPM (engineRpm)
   📈 Data Availability: 42.3% (1,889,794 readings)
   🎯 DPF Relevance: RPM affects DPF regeneration efficiency
   📏 Normal Range: (800, 1800) RPM
   📊 Current Range: 26.0 - 2618.0 RPM

📊 DEF (Diesel Exhaust Fluid) Level (defLevelMilliPercent)
   📈 Data Availability: 45.9% (2,048,969 readings)
   🎯 DPF Relevance: Low DEF prevents proper DPF operation
   📏 Normal Range: (60000, 95000) milli-%
   📊 Current Range: 400.0 - 100000.0 milli-%

📊 Engine Coolant Temperature (engineCoolantTemperatureMilliC)
   📈 Data Availability: 33.5% (1,497,692 readings)
   🎯 DPF Relevance: Temperature affects DPF regeneration cycles
   📏 Normal Range: (75000, 90000) milli-°C
   📊 Current Range: 1000.0 - 123000.0 milli-°C

📊 Vehic

In [11]:
def calculate_trend_features(sensor_data, sensor_name, time_col='time'):
    """
    Calculate interpretable trend features for a sensor.
    Returns features that fleet managers can understand and act upon.
    """
    if len(sensor_data) < 3:  # Need minimum points for trend
        return {}
    
    # Prepare data
    data = sensor_data.dropna().sort_values(time_col)
    if len(data) < 3:
        return {}
    
    values = data[sensor_name].values
    times = pd.to_numeric(data[time_col]) / 1e9  # Convert to seconds
    
    features = {}
    
    # 1. LINEAR TREND (Most Important)
    # Calculate slope in original units per day
    slope_per_second, intercept = np.polyfit(times, values, 1)
    slope_per_day = slope_per_second * 86400  # Convert to per day
    
    features[f'{sensor_name}_trend_slope_per_day'] = slope_per_day
    
    # 2. TREND STRENGTH (How reliable is the trend?)
    # R-squared shows how well the linear trend fits
    correlation = np.corrcoef(times, values)[0, 1]
    r_squared = correlation ** 2 if not np.isnan(correlation) else 0
    features[f'{sensor_name}_trend_strength'] = r_squared
    
    # 3. TREND SIGNIFICANCE (Is this trend meaningful?)
    # Calculate statistical significance of the trend
    if len(values) >= 3:
        slope, intercept, r_value, p_value, std_err = stats.linregress(times, values)
        features[f'{sensor_name}_trend_p_value'] = p_value
        features[f'{sensor_name}_trend_significant'] = 1 if p_value < 0.05 else 0
    
    # 4. RECENT VS HISTORICAL TREND
    # Compare recent behavior to historical (last 25% vs first 75%)
    split_point = int(len(values) * 0.75)
    if split_point > 2 and split_point < len(values) - 2:
        historical_mean = np.mean(values[:split_point])
        recent_mean = np.mean(values[split_point:])
        
        if historical_mean != 0:
            percent_change = ((recent_mean - historical_mean) / abs(historical_mean)) * 100
            features[f'{sensor_name}_recent_vs_historical_pct'] = percent_change
    
    # 5. VOLATILITY TREND (Is stability changing?)
    # Calculate if the sensor is becoming more or less stable
    if len(values) >= 6:
        mid_point = len(values) // 2
        early_std = np.std(values[:mid_point])
        late_std = np.std(values[mid_point:])
        
        if early_std > 0:
            volatility_change = (late_std - early_std) / early_std
            features[f'{sensor_name}_volatility_change'] = volatility_change
    
    return features

# Demonstrate trend analysis with a sample vehicle
print("🔍 Demonstrating Trend Analysis")
print("="*40)

# Get a vehicle with good data coverage
vehicle_counts = sensor_df['vin'].value_counts()
if len(vehicle_counts) > 0:
    sample_vin = vehicle_counts.index[0]
    sample_data = sensor_df[sensor_df['vin'] == sample_vin].copy()
    
    print(f"Sample Vehicle: {sample_vin}")
    print(f"Data Points: {len(sample_data)}")
    print(f"Date Range: {sample_data['time'].min()} to {sample_data['time'].max()}")
    
    # Calculate trends for each DPF sensor
    all_trends = {}
    for sensor in dpf_sensors.keys():
        if sensor in sample_data.columns:
            sensor_subset = sample_data[['time', sensor]].dropna()
            if len(sensor_subset) >= 5:
                trends = calculate_trend_features(sensor_subset, sensor)
                all_trends.update(trends)
                
                # Show human-readable interpretation
                slope_key = f'{sensor}_trend_slope_per_day'
                if slope_key in trends:
                    slope = trends[slope_key]
                    direction = "increasing" if slope > 0 else "decreasing"
                    strength = trends.get(f'{sensor}_trend_strength', 0)
                    
                    print(f"\n📈 {dpf_sensors[sensor]['description']}:")
                    print(f"   Trend: {direction} by {abs(slope):.3f} {dpf_sensors[sensor]['unit']}/day")
                    print(f"   Strength: {strength:.3f} (0=weak, 1=strong linear trend)")
                    
                    # Interpret for fleet manager
                    if abs(slope) > 0.1 and strength > 0.3:
                        print(f"   🚨 Alert: Strong {direction} trend detected!")
                    elif abs(slope) > 0.05:
                        print(f"   ⚠️ Caution: Moderate {direction} trend")
                    else:
                        print(f"   ✅ Normal: Stable readings")
    
    print(f"\n📊 Total trend features calculated: {len(all_trends)}")
else:
    print("❌ No vehicle data available for trend analysis")

🔍 Demonstrating Trend Analysis
Sample Vehicle: 1XK1D40X1NJ495537
Data Points: 269964
Date Range: 2024-06-08 00:03:00 to 2025-05-21 20:40:00

📈 Engine Load Percentage:
   Trend: increasing by 0.013 %/day
   Strength: 0.001 (0=weak, 1=strong linear trend)
   ✅ Normal: Stable readings

📈 Engine RPM:
   Trend: increasing by 0.111 RPM/day
   Strength: 0.001 (0=weak, 1=strong linear trend)
   ⚠️ Caution: Moderate increasing trend

📈 DEF (Diesel Exhaust Fluid) Level:
   Trend: decreasing by 61.047 milli-%/day
   Strength: 0.072 (0=weak, 1=strong linear trend)
   ⚠️ Caution: Moderate decreasing trend

📈 Engine Coolant Temperature:
   Trend: decreasing by 9.565 milli-°C/day
   Strength: 0.005 (0=weak, 1=strong linear trend)
   ⚠️ Caution: Moderate decreasing trend

📈 Vehicle Speed:
   Trend: decreasing by 0.003 mph/day
   Strength: 0.000 (0=weak, 1=strong linear trend)
   ✅ Normal: Stable readings

📊 Total trend features calculated: 30


In [12]:
def calculate_threshold_features(sensor_data, sensor_name, thresholds, time_col='time'):
    """
    Calculate threshold-based features that are easy to interpret and act upon.
    """
    if len(sensor_data) < 2:
        return {}
    
    # Prepare data
    data = sensor_data.dropna().sort_values(time_col)
    if len(data) < 2:
        return {}
    
    values = data[sensor_name].values
    features = {}
    
    # 1. PERCENTAGE OF TIME OUTSIDE THRESHOLDS
    
    # Time above high threshold
    if 'high' in thresholds:
        high_thresh = thresholds['high']
        pct_above_high = (values > high_thresh).mean() * 100
        features[f'{sensor_name}_pct_time_above_high'] = pct_above_high
    
    # Time below low threshold
    if 'low' in thresholds:
        low_thresh = thresholds['low']
        pct_below_low = (values < low_thresh).mean() * 100
        features[f'{sensor_name}_pct_time_below_low'] = pct_below_low
    
    # Total time outside normal range
    if 'high' in thresholds and 'low' in thresholds:
        outside_normal = ((values > thresholds['high']) | (values < thresholds['low'])).mean() * 100
        features[f'{sensor_name}_pct_time_outside_normal'] = outside_normal
    
    # 2. CONSECUTIVE VIOLATIONS (Streak Analysis)
    
    # Longest streak above high threshold
    if 'high' in thresholds:
        above_high = values > thresholds['high']
        max_streak_high = calculate_max_consecutive(above_high)
        features[f'{sensor_name}_max_streak_above_high'] = max_streak_high
    
    # Longest streak below low threshold
    if 'low' in thresholds:
        below_low = values < thresholds['low']
        max_streak_low = calculate_max_consecutive(below_low)
        features[f'{sensor_name}_max_streak_below_low'] = max_streak_low
    
    # 3. FREQUENCY OF VIOLATIONS
    
    # How many separate incidents of threshold violations?
    if 'high' in thresholds:
        high_violations = count_violation_episodes(values > thresholds['high'])
        features[f'{sensor_name}_high_violation_episodes'] = high_violations
    
    if 'low' in thresholds:
        low_violations = count_violation_episodes(values < thresholds['low'])
        features[f'{sensor_name}_low_violation_episodes'] = low_violations
    
    # 4. SEVERITY OF VIOLATIONS
    
    # Average severity when violations occur
    if 'high' in thresholds:
        high_violations_mask = values > thresholds['high']
        if high_violations_mask.any():
            avg_excess = (values[high_violations_mask] - thresholds['high']).mean()
            features[f'{sensor_name}_avg_excess_above_high'] = avg_excess
    
    if 'low' in thresholds:
        low_violations_mask = values < thresholds['low']
        if low_violations_mask.any():
            avg_deficit = (thresholds['low'] - values[low_violations_mask]).mean()
            features[f'{sensor_name}_avg_deficit_below_low'] = avg_deficit
    
    return features

def calculate_max_consecutive(boolean_array):
    """Calculate maximum consecutive True values in a boolean array."""
    if len(boolean_array) == 0:
        return 0
    
    max_streak = 0
    current_streak = 0
    
    for value in boolean_array:
        if value:
            current_streak += 1
            max_streak = max(max_streak, current_streak)
        else:
            current_streak = 0
    
    return max_streak

def count_violation_episodes(boolean_array):
    """Count number of separate violation episodes (groups of consecutive True values)."""
    if len(boolean_array) == 0:
        return 0
    
    episodes = 0
    in_violation = False
    
    for value in boolean_array:
        if value and not in_violation:
            episodes += 1
            in_violation = True
        elif not value:
            in_violation = False
    
    return episodes

# Demonstrate threshold analysis
print("🎯 Demonstrating Threshold Analysis")
print("="*40)

if len(vehicle_counts) > 0:
    sample_vin = vehicle_counts.index[0]
    sample_data = sensor_df[sensor_df['vin'] == sample_vin].copy()
    
    print(f"Analyzing thresholds for Vehicle: {sample_vin}")
    
    # Calculate threshold features for each sensor
    all_threshold_features = {}
    
    for sensor, info in dpf_sensors.items():
        if sensor in sample_data.columns:
            sensor_subset = sample_data[['time', sensor]].dropna()
            if len(sensor_subset) >= 5:
                thresholds = info['alert_thresholds']
                threshold_features = calculate_threshold_features(sensor_subset, sensor, thresholds)
                all_threshold_features.update(threshold_features)
                
                print(f"\n🎯 {info['description']} ({sensor}):")
                
                # Show key threshold metrics
                pct_high_key = f'{sensor}_pct_time_above_high'
                pct_low_key = f'{sensor}_pct_time_below_low'
                
                if pct_high_key in threshold_features:
                    pct_high = threshold_features[pct_high_key]
                    print(f"   ⬆️ Time above {thresholds['high']} {info['unit']}: {pct_high:.1f}%")
                    
                    if pct_high > 20:
                        print(f"      🚨 ALERT: Excessive high readings!")
                    elif pct_high > 10:
                        print(f"      ⚠️ WARNING: Frequent high readings")
                    else:
                        print(f"      ✅ Normal high threshold behavior")
                
                if pct_low_key in threshold_features:
                    pct_low = threshold_features[pct_low_key]
                    print(f"   ⬇️ Time below {thresholds['low']} {info['unit']}: {pct_low:.1f}%")
                    
                    if pct_low > 20:
                        print(f"      🚨 ALERT: Excessive low readings!")
                    elif pct_low > 10:
                        print(f"      ⚠️ WARNING: Frequent low readings")
                    else:
                        print(f"      ✅ Normal low threshold behavior")
                
                # Show streak information
                streak_high_key = f'{sensor}_max_streak_above_high'
                if streak_high_key in threshold_features:
                    streak = threshold_features[streak_high_key]
                    print(f"   📊 Longest high streak: {streak} consecutive readings")
    
    print(f"\n📊 Total threshold features calculated: {len(all_threshold_features)}")
else:
    print("❌ No vehicle data available for threshold analysis")

🎯 Demonstrating Threshold Analysis
Analyzing thresholds for Vehicle: 1XK1D40X1NJ495537

🎯 Engine Load Percentage (engineLoadPercent):
   ⬆️ Time above 80 %: 24.8%
      🚨 ALERT: Excessive high readings!
   ⬇️ Time below 10 %: 6.4%
      ✅ Normal low threshold behavior
   📊 Longest high streak: 16 consecutive readings

🎯 Engine RPM (engineRpm):
   ⬆️ Time above 2000 RPM: 0.0%
      ✅ Normal high threshold behavior
   ⬇️ Time below 600 RPM: 4.8%
      ✅ Normal low threshold behavior
   📊 Longest high streak: 1 consecutive readings

🎯 DEF (Diesel Exhaust Fluid) Level (defLevelMilliPercent):
   ⬆️ Time above 95000 milli-%: 5.4%
      ✅ Normal high threshold behavior
   ⬇️ Time below 50000 milli-%: 28.5%
      🚨 ALERT: Excessive low readings!
   📊 Longest high streak: 101 consecutive readings

🎯 Engine Coolant Temperature (engineCoolantTemperatureMilliC):
   ⬆️ Time above 95000 milli-°C: 17.6%
      ⚠️ WARNING: Frequent high readings
   ⬇️ Time below 65000 milli-°C: 4.2%
      ✅ Normal low 

In [13]:
def calculate_operational_pattern_features(sensor_data, time_col='time'):
    """
    Calculate operational pattern features that help identify changes in vehicle usage.
    These features help fleet managers understand if operational changes are affecting DPF health.
    """
    if len(sensor_data) < 10:  # Need reasonable amount of data for patterns
        return {}
    
    features = {}
    
    # 1. DUTY CYCLE ANALYSIS (How hard is the vehicle working?)
    if 'engineLoadPercent' in sensor_data.columns:
        load_data = sensor_data['engineLoadPercent'].dropna()
        if len(load_data) >= 10:
            
            # Average duty cycle
            avg_load = load_data.mean()
            features['avg_engine_load'] = avg_load
            
            # Load distribution analysis
            features['pct_time_high_load'] = (load_data > 70).mean() * 100  # >70% load
            features['pct_time_medium_load'] = ((load_data >= 40) & (load_data <= 70)).mean() * 100
            features['pct_time_low_load'] = (load_data < 40).mean() * 100  # <40% load
            
            # Load consistency (coefficient of variation)
            if avg_load > 0:
                features['load_consistency'] = load_data.std() / avg_load
    
    # 2. DRIVE PROFILE ANALYSIS (City vs Highway driving)
    if 'ecuSpeedMph' in sensor_data.columns:
        speed_data = sensor_data['ecuSpeedMph'].dropna()
        if len(speed_data) >= 10:
            
            # Speed distribution (proxy for drive cycle)
            features['pct_time_highway_speed'] = (speed_data > 45).mean() * 100  # >45 mph = highway
            features['pct_time_city_speed'] = ((speed_data > 0) & (speed_data <= 35)).mean() * 100  # 0-35 mph = city
            features['pct_time_stopped'] = (speed_data == 0).mean() * 100  # Stopped
            
            # Average operating speed
            features['avg_operating_speed'] = speed_data[speed_data > 0].mean() if (speed_data > 0).any() else 0
            
            # Speed variability (smooth vs stop-and-go driving)
            features['speed_variability'] = speed_data.std()
    
    # 3. OPERATIONAL TIMING PATTERNS
    if time_col in sensor_data.columns:
        # Add hour of day for operational pattern analysis
        sensor_data_copy = sensor_data.copy()
        sensor_data_copy['hour'] = pd.to_datetime(sensor_data_copy[time_col]).dt.hour
        
        # Operating hours distribution
        operating_hours = sensor_data_copy['hour'].value_counts().sort_index()
        
        # Peak operating hours
        if len(operating_hours) > 0:
            features['peak_operating_hour'] = operating_hours.idxmax()
            features['operating_hour_spread'] = operating_hours.max() - operating_hours.min()
    
    # 4. MULTI-SENSOR OPERATIONAL PATTERNS
    # Correlation between load and speed (operational efficiency)
    if 'engineLoadPercent' in sensor_data.columns and 'ecuSpeedMph' in sensor_data.columns:
        load_speed_data = sensor_data[['engineLoadPercent', 'ecuSpeedMph']].dropna()
        if len(load_speed_data) >= 10:
            correlation = load_speed_data['engineLoadPercent'].corr(load_speed_data['ecuSpeedMph'])
            features['load_speed_correlation'] = correlation if not np.isnan(correlation) else 0
    
    # Engine efficiency patterns (RPM vs Load)
    if 'engineRpm' in sensor_data.columns and 'engineLoadPercent' in sensor_data.columns:
        rpm_load_data = sensor_data[['engineRpm', 'engineLoadPercent']].dropna()
        if len(rpm_load_data) >= 10:
            # Calculate efficiency proxy (load per RPM)
            efficiency_data = rpm_load_data[rpm_load_data['engineRpm'] > 0].copy()
            if len(efficiency_data) > 0:
                efficiency_data['load_per_rpm'] = efficiency_data['engineLoadPercent'] / efficiency_data['engineRpm'] * 1000
                features['avg_load_per_rpm'] = efficiency_data['load_per_rpm'].mean()
    
    return features

def compare_operational_periods(sensor_data, split_ratio=0.7, time_col='time'):
    """
    Compare operational patterns between historical (early) and recent periods.
    This helps identify if operational changes are affecting DPF health.
    """
    if len(sensor_data) < 20:  # Need enough data to split meaningfully
        return {}
    
    # Sort by time and split
    sorted_data = sensor_data.sort_values(time_col)
    split_point = int(len(sorted_data) * split_ratio)
    
    historical_data = sorted_data.iloc[:split_point]
    recent_data = sorted_data.iloc[split_point:]
    
    # Calculate operational features for both periods
    historical_features = calculate_operational_pattern_features(historical_data, time_col)
    recent_features = calculate_operational_pattern_features(recent_data, time_col)
    
    # Calculate changes
    changes = {}
    for feature in historical_features:
        if feature in recent_features:
            historical_value = historical_features[feature]
            recent_value = recent_features[feature]
            
            if historical_value != 0:
                percent_change = ((recent_value - historical_value) / abs(historical_value)) * 100
                changes[f'{feature}_change_pct'] = percent_change
    
    return changes

# Demonstrate operational pattern analysis
print("🔄 Demonstrating Operational Pattern Analysis")
print("="*50)

if len(vehicle_counts) > 0:
    sample_vin = vehicle_counts.index[0]
    sample_data = sensor_df[sensor_df['vin'] == sample_vin].copy()
    
    print(f"Analyzing operational patterns for Vehicle: {sample_vin}")
    print(f"Data points: {len(sample_data)}")
    
    # Calculate current operational features
    operational_features = calculate_operational_pattern_features(sample_data)
    
    if operational_features:
        print(f"\n📊 Current Operational Profile:")
        
        # Duty cycle analysis
        if 'avg_engine_load' in operational_features:
            avg_load = operational_features['avg_engine_load']
            print(f"   🔧 Average Engine Load: {avg_load:.1f}%")
            
            if avg_load > 60:
                print(f"      🚨 High duty cycle - may stress DPF")
            elif avg_load < 30:
                print(f"      ⚠️ Low duty cycle - may not regenerate DPF properly")
            else:
                print(f"      ✅ Moderate duty cycle")
        
        # Drive profile analysis
        if 'pct_time_highway_speed' in operational_features:
            highway_pct = operational_features['pct_time_highway_speed']
            city_pct = operational_features.get('pct_time_city_speed', 0)
            
            print(f"   🛣️ Highway driving: {highway_pct:.1f}% of time")
            print(f"   🏙️ City driving: {city_pct:.1f}% of time")
            
            if highway_pct < 20:
                print(f"      🚨 Mostly city driving - poor for DPF regeneration")
            elif highway_pct > 60:
                print(f"      ✅ Good highway driving - helps DPF regeneration")
            else:
                print(f"      ⚠️ Mixed driving profile")
    
    # Compare historical vs recent patterns
    pattern_changes = compare_operational_periods(sample_data)
    
    if pattern_changes:
        print(f"\n📈 Operational Changes (Recent vs Historical):")
        
        significant_changes = {k: v for k, v in pattern_changes.items() if abs(v) > 10}
        
        if significant_changes:
            for change_feature, percent_change in significant_changes.items():
                base_feature = change_feature.replace('_change_pct', '')
                direction = "increased" if percent_change > 0 else "decreased"
                print(f"   📊 {base_feature}: {direction} by {abs(percent_change):.1f}%")
                
                # Interpret the change
                if abs(percent_change) > 25:
                    print(f"      🚨 Significant operational change detected!")
                elif abs(percent_change) > 15:
                    print(f"      ⚠️ Notable operational change")
        else:
            print(f"   ✅ Stable operational patterns")
    
    total_features = len(operational_features) + len(pattern_changes)
    print(f"\n📊 Total operational features calculated: {total_features}")
else:
    print("❌ No vehicle data available for operational pattern analysis")

🔄 Demonstrating Operational Pattern Analysis
Analyzing operational patterns for Vehicle: 1XK1D40X1NJ495537
Data points: 269964

📊 Current Operational Profile:
   🔧 Average Engine Load: 51.1%
      ✅ Moderate duty cycle
   🛣️ Highway driving: 73.6% of time
   🏙️ City driving: 17.2% of time
      ✅ Good highway driving - helps DPF regeneration

📈 Operational Changes (Recent vs Historical):
   📊 peak_operating_hour: increased by 16.7%
      ⚠️ Notable operational change
   📊 operating_hour_spread: decreased by 57.4%
      🚨 Significant operational change detected!

📊 Total operational features calculated: 28


In [14]:
def build_comprehensive_feature_dataset(maintenance_df, sensor_df, window_days=30):
    """
    Build a comprehensive dataset with all our interpretable features.
    This combines trend, threshold, and operational pattern features.
    """
    print(f"🔧 Building comprehensive feature dataset...")
    
    feature_data = []
    
    for _, maintenance_row in maintenance_df.iterrows():
        vin = maintenance_row['VIN Number']
        maintenance_date = maintenance_row['Date of Issue']
        
        if pd.isna(vin) or pd.isna(maintenance_date):
            continue
        
        # Get sensor data before maintenance
        start_date = maintenance_date - timedelta(days=window_days)
        
        vehicle_data = sensor_df[
            (sensor_df['vin'] == vin) &
            (sensor_df['time'] >= start_date) &
            (sensor_df['time'] < maintenance_date)
        ].copy()
        
        if len(vehicle_data) < 10:  # Need minimum data
            continue
        
        # Calculate all feature types
        all_features = {
            'vin': vin,
            'vehicle_number': maintenance_row['Vehicle_Number'],
            'maintenance_date': maintenance_date,
            'maintenance_type': maintenance_row['lines_jobDescriptions'],
            'data_points': len(vehicle_data),
            'window_days': window_days
        }
        
        # 1. Trend features for each DPF sensor
        for sensor in dpf_sensors.keys():
            if sensor in vehicle_data.columns:
                sensor_subset = vehicle_data[['time', sensor]].dropna()
                if len(sensor_subset) >= 5:
                    trend_features = calculate_trend_features(sensor_subset, sensor)
                    all_features.update(trend_features)
        
        # 2. Threshold features for each DPF sensor
        for sensor, info in dpf_sensors.items():
            if sensor in vehicle_data.columns:
                sensor_subset = vehicle_data[['time', sensor]].dropna()
                if len(sensor_subset) >= 5:
                    threshold_features = calculate_threshold_features(
                        sensor_subset, sensor, info['alert_thresholds']
                    )
                    all_features.update(threshold_features)
        
        # 3. Operational pattern features
        operational_features = calculate_operational_pattern_features(vehicle_data)
        all_features.update(operational_features)
        
        # 4. Operational change features
        pattern_changes = compare_operational_periods(vehicle_data)
        all_features.update(pattern_changes)
        
        feature_data.append(all_features)
    
    return pd.DataFrame(feature_data)

# Build the comprehensive feature dataset
print("🔧 Building Comprehensive Interpretable Feature Dataset")
print("="*60)

feature_dataset = build_comprehensive_feature_dataset(maintenance_df, sensor_df)

if len(feature_dataset) > 0:
    print(f"✅ Built feature dataset with {len(feature_dataset)} examples")
    
    # Analyze feature coverage
    feature_cols = [col for col in feature_dataset.columns if col not in [
        'vin', 'vehicle_number', 'maintenance_date', 'maintenance_type', 'data_points', 'window_days'
    ]]
    
    print(f"📊 Total interpretable features: {len(feature_cols)}")
    
    # Categorize features by type
    trend_features = [col for col in feature_cols if 'trend' in col]
    threshold_features = [col for col in feature_cols if any(x in col for x in ['pct_time', 'streak', 'violation', 'excess', 'deficit'])]
    operational_features = [col for col in feature_cols if col not in trend_features + threshold_features]
    
    print(f"   📈 Trend features: {len(trend_features)}")
    print(f"   🎯 Threshold features: {len(threshold_features)}")
    print(f"   🔄 Operational features: {len(operational_features)}")
    
    # Show maintenance type distribution
    print(f"\n🔧 Maintenance Type Distribution:")
    for maint_type, count in feature_dataset['maintenance_type'].value_counts().items():
        print(f"   {maint_type}: {count} examples")
        
else:
    print("❌ Could not build feature dataset - insufficient data")

🔧 Building Comprehensive Interpretable Feature Dataset
🔧 Building comprehensive feature dataset...
✅ Built feature dataset with 46 examples
📊 Total interpretable features: 102
   📈 Trend features: 20
   🎯 Threshold features: 56
   🔄 Operational features: 26

🔧 Maintenance Type Distribution:
   EXHAUST SYSTEM INSPECT DIAGNOSE: 26 examples
   EXHAUST SYSTEM: 14 examples
   FILTER - DIESEL PARTICULATE: 6 examples


In [15]:
# Validate feature quality and predictive power
def validate_interpretable_features(feature_dataset):
    """
    Validate the quality and predictive power of our interpretable features.
    """
    if len(feature_dataset) == 0:
        return
    
    print("🧪 Validating Interpretable Features")
    print("="*40)
    
    # Get feature columns
    feature_cols = [col for col in feature_dataset.columns if col not in [
        'vin', 'vehicle_number', 'maintenance_date', 'maintenance_type', 'data_points', 'window_days'
    ]]
    
    if len(feature_cols) == 0:
        print("❌ No features to validate")
        return
    
    # 1. DATA QUALITY VALIDATION
    print("📊 Data Quality Analysis:")
    
    # Check for missing values
    missing_rates = feature_dataset[feature_cols].isnull().mean() * 100
    high_missing = missing_rates[missing_rates > 50]
    
    if len(high_missing) > 0:
        print(f"   ⚠️ Features with >50% missing data: {len(high_missing)}")
        for feature, missing_pct in high_missing.head(5).items():
            print(f"      - {feature}: {missing_pct:.1f}% missing")
    else:
        print(f"   ✅ Good data coverage - no features with >50% missing data")
    
    # 2. FEATURE DISCRIMINATIVE POWER
    print(f"\n🔍 Feature Discriminative Power (by Maintenance Type):")
    
    # For each maintenance type, find features that discriminate well
    maintenance_types = feature_dataset['maintenance_type'].unique()
    
    if len(maintenance_types) > 1:
        # Calculate feature means by maintenance type
        type_means = feature_dataset.groupby('maintenance_type')[feature_cols].mean()
        
        # Calculate coefficient of variation across maintenance types for each feature
        discriminative_power = {}
        for feature in feature_cols:
            values = type_means[feature].dropna()
            if len(values) > 1 and values.std() > 0:
                cv = values.std() / abs(values.mean()) if values.mean() != 0 else 0
                discriminative_power[feature] = cv
        
        # Show top discriminative features
        if discriminative_power:
            top_discriminative = sorted(discriminative_power.items(), key=lambda x: x[1], reverse=True)[:10]
            
            print(f"   🏆 Top 10 Most Discriminative Features:")
            for i, (feature, cv) in enumerate(top_discriminative, 1):
                # Determine feature type
                if 'trend' in feature:
                    feature_type = "📈 Trend"
                elif any(x in feature for x in ['pct_time', 'streak', 'violation']):
                    feature_type = "🎯 Threshold"
                else:
                    feature_type = "🔄 Operational"
                
                print(f"      {i}. {feature_type} - {feature}: CV={cv:.3f}")
    
    # 3. FEATURE INTERPRETABILITY ASSESSMENT
    print(f"\n💡 Feature Interpretability Assessment:")
    
    interpretability_scores = {
        'trend_slope': {'score': 10, 'explanation': 'Clear direction and rate of change'},
        'pct_time': {'score': 9, 'explanation': 'Percentage time outside thresholds'},
        'avg_': {'score': 8, 'explanation': 'Simple average values'},
        'max_streak': {'score': 8, 'explanation': 'Consecutive violations count'},
        'change_pct': {'score': 7, 'explanation': 'Percentage change over time'},
        'correlation': {'score': 6, 'explanation': 'Relationship between sensors'},
        'volatility': {'score': 5, 'explanation': 'Stability measure'}
    }
    
    # Score each feature
    feature_scores = []
    for feature in feature_cols:
        score = 3  # Default score
        explanation = "General metric"
        
        for pattern, info in interpretability_scores.items():
            if pattern in feature:
                score = info['score']
                explanation = info['explanation']
                break
        
        feature_scores.append((feature, score, explanation))
    
    # Show most interpretable features
    feature_scores.sort(key=lambda x: x[1], reverse=True)
    
    print(f"   🌟 Most Interpretable Features (Score 8-10):")
    high_interpretability = [f for f in feature_scores if f[1] >= 8]
    
    for feature, score, explanation in high_interpretability[:10]:
        print(f"      • {feature} (Score: {score}/10) - {explanation}")
    
    # 4. FEATURE RECOMMENDATION
    print(f"\n🎯 Feature Recommendations for Fleet Managers:")
    
    # Combine discriminative power and interpretability
    if discriminative_power:
        recommended_features = []
        
        for feature, interp_score, explanation in feature_scores:
            if feature in discriminative_power and interp_score >= 7:
                combined_score = discriminative_power[feature] * (interp_score / 10)
                recommended_features.append((feature, combined_score, interp_score, explanation))
        
        recommended_features.sort(key=lambda x: x[1], reverse=True)
        
        print(f"   🏆 Top 5 Recommended Features (High interpretability + High predictive power):")
        for i, (feature, combined_score, interp_score, explanation) in enumerate(recommended_features[:5], 1):
            print(f"      {i}. {feature}")
            print(f"         Interpretability: {interp_score}/10 - {explanation}")
            print(f"         Combined Score: {combined_score:.3f}")
            print()

# Run feature validation
if len(feature_dataset) > 0:
    validate_interpretable_features(feature_dataset)
else:
    print("❌ No feature dataset available for validation")

🧪 Validating Interpretable Features
📊 Data Quality Analysis:
   ⚠️ Features with >50% missing data: 1
      - ecuSpeedMph_avg_excess_above_high: 76.1% missing

🔍 Feature Discriminative Power (by Maintenance Type):
   🏆 Top 10 Most Discriminative Features:
      1. 📈 Trend - engineCoolantTemperatureMilliC_trend_slope_per_day: CV=24.380
      2. 🎯 Threshold - pct_time_city_speed_change_pct: CV=20.160
      3. 📈 Trend - defLevelMilliPercent_trend_slope_per_day: CV=17.688
      4. 🔄 Operational - load_speed_correlation_change_pct: CV=16.439
      5. 🔄 Operational - avg_engine_load_change_pct: CV=13.777
      6. 🎯 Threshold - pct_time_medium_load_change_pct: CV=13.562
      7. 📈 Trend - engineRpm_trend_slope_per_day: CV=4.648
      8. 🎯 Threshold - pct_time_stopped_change_pct: CV=4.272
      9. 🔄 Operational - ecuSpeedMph_recent_vs_historical_pct: CV=3.999
      10. 📈 Trend - ecuSpeedMph_trend_slope_per_day: CV=2.841

💡 Feature Interpretability Assessment:
   🌟 Most Interpretable Features (

In [16]:
# Save the interpretable feature dataset for use in other notebooks
if len(feature_dataset) > 0:
    output_file = '../data/interpretable_features_dataset.csv'
    feature_dataset.to_csv(output_file, index=False)
    
    print("💾 FEATURE ENGINEERING COMPLETE!")
    print("="*50)
    print(f"✅ Saved interpretable features to: {output_file}")
    print(f"📊 Dataset contains: {len(feature_dataset)} examples")
    
    feature_cols = [col for col in feature_dataset.columns if col not in [
        'vin', 'vehicle_number', 'maintenance_date', 'maintenance_type', 'data_points', 'window_days'
    ]]
    print(f"🔧 Total features: {len(feature_cols)}")
    
    # Feature breakdown
    trend_features = [col for col in feature_cols if 'trend' in col]
    threshold_features = [col for col in feature_cols if any(x in col for x in ['pct_time', 'streak', 'violation'])]
    operational_features = [col for col in feature_cols if col not in trend_features + threshold_features]
    
    print(f"   📈 Trend features: {len(trend_features)}")
    print(f"   🎯 Threshold features: {len(threshold_features)}")
    print(f"   🔄 Operational features: {len(operational_features)}")
    
    print(f"\n🎉 Ready for model building and deployment!")
    print(f"Next step: Use these features in the explainable model notebook")
else:
    print("❌ No interpretable features dataset created")
    print("Please check data availability and try again")

print(f"\n📚 What You've Learned:")
print(f"• How to create trend features that show sensor degradation patterns")
print(f"• How to build threshold features for actionable alerts")
print(f"• How to detect operational pattern changes affecting DPF health")
print(f"• How to validate feature quality and interpretability")
print(f"• Best practices for explainable feature engineering")

💾 FEATURE ENGINEERING COMPLETE!
✅ Saved interpretable features to: ../data/interpretable_features_dataset.csv
📊 Dataset contains: 46 examples
🔧 Total features: 102
   📈 Trend features: 20
   🎯 Threshold features: 47
   🔄 Operational features: 35

🎉 Ready for model building and deployment!
Next step: Use these features in the explainable model notebook

📚 What You've Learned:
• How to create trend features that show sensor degradation patterns
• How to build threshold features for actionable alerts
• How to detect operational pattern changes affecting DPF health
• How to validate feature quality and interpretability
• Best practices for explainable feature engineering
